In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내셨어요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QA5Yp5zsdF3HRQ6P7lCE9ph1GrX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bcb12bf3-45a6-4c86-a23b-d018635517b0-0', usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [26]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [27]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n0NHOK08cnEmeW3K5Qv4sEAt', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C3QA6PVGERiMHVvt6RbkhIeik4tYE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4d5b23a8-a30d-4c41-a652-561fb85eb138-0', tool_ca

In [28]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2025-08-12 01:59:47 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n0NHOK08cnEmeW3K5Qv4sEAt', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C3QA6PVGERiMHVvt6RbkhIeik4tYE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4d5b23a8-a30d-4c41-a652-561fb85eb138-0', tool_

In [29]:
llm_with_tools.invoke(messages)

AIMessage(content='현재 부산은 2025년 8월 12일 01시 59분 47초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 192, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C3QA7vy0ir5FEnQL1vmC6I9vDqXUP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--09bff703-98ba-4068-a944-e7c7b69e6940-0', usage_metadata={'input_tokens': 192, 'output_tokens': 25, 'total_tokens': 217, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [30]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [31]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [32]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_7D2kuykPwelpVHu0D23kPdCL', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QA8aTj7NFMp0vlSPj7Sq0VMWMal', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--c89ff9e5-f83d-4065-a169-1a64ba80579b-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_7D2kuykPwelpVHu0D23kPdCL', 'type': 'tool_call'}] usage_metadata={'inp

In [33]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-07-11 00:00:00-04:00 | 307.89 | 314.09 | 305.65 | 313.51  | 7.92364e+07 |           0 |              0 |\n| 2025-07-14 00:00:00-04:00 | 317.73 | 322.6  | 312.67 | 316.9   | 7.80434e+07 |           0 |              0 |\n| 2025-07-15 00:00:00-04:00 | 319.68 | 321.2  | 310.5  | 310.78  | 7.75563e+07 |           0 |              0 |\n| 2025-07-16 00:00:00-04:00 | 312.8  | 323.5  | 312.62 | 321.67  | 9.72848e+07 |           0 |              0 |\n| 2025-07-17 00:00:00-04:00 | 323.15 | 324.34 | 317.06 | 319.41  | 7.39229e+07 |           0 |              0 |\n| 2025-07-18 00:00:00-04:00 | 321.66 | 330.9  | 321.42 | 329.65  | 9.4255e+07  |           0 |              0 |\n| 2025-07-21 00:00:00-04:0

In [34]:
llm_with_tools.invoke(messages)

AIMessage(content='테슬라의 현재 주가는 2025년 8월 11일 기준으로 344.30 달러입니다. 한 달 전인 2025년 7월 11일의 종가는 313.51 달러였습니다.\n\n따라서, 테슬라의 주가는 한 달 전에 비해 상승하였습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1638, 'total_tokens': 1710, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QAAPsZgNITrrMrPJ0RU2CyYDDHf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1505df3c-6ea3-4f75-a112-5289f428bb82-0', usage_metadata={'input_tokens': 1638, 'output_tokens': 72, 'total_tokens': 1710, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 08-4 스트림 출력

In [35]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]):
    print(c.content, end='|') 

|안|녕하세요|!| 한국| 사회|의| 문제|점|에| 대해서|는| 여러| 가지|가| 있습니다|.| 여기| 몇| 가지| 주요| 문제|를| 소개|해| 드|릴|게|요|.

|1|.| **|청|년| 실|업|**|:| 한국|의| 청|년| 실|업|률|은| 다른| 국가|에| 비|해| 높은| 편|입니다|.| 취|업| 경쟁|이| 치|열|하고|,| 안정|적인| 일|자|리를| 찾|기가| 어려|운| 점|이| 많은| 청|년|들에게| 큰| 부담|으로| 작|용|하고| 있습니다|.

|2|.| **|주|거| 문제|**|:| 서울|과| 같은| 대|도|시|에서는| 높은| 집|값|과| 전|세| 가격|으로| 인해| 많은| 사람들이| 주|거| 불|안을| 겪|고| 있습니다|.| 특히| 청|년|층|과| 신|혼|부|부|들이| 적|절|한| 주|거|지를| 찾|기| 어려|운| 상황|입니다|.

|3|.| **|교육| 문제|**|:| 과|도|한| 학|업| 부담|과| 입|시| 경쟁|으로| 인해| 학생|들이| 심|리|적| 스트|레|스를| 많이| 받고| 있습니다|.| 또한| 사|교육|에| 대한| 의|존|도가| 높|아|져| 교육|의| 불|평|등|도| 심|화|되고| 있습니다|.

|4|.| **|노|인| 문제|**|:| 고|령|화| 사회|로| 접|어|들|면서| 노|인| 빈|곤|과| 고|독| 문제가| 심|각|해|지고| 있습니다|.| 많은| 노|인|들이| 사회| 안전|망|에서| 소|외|되|거나|,| 경제|적으로| 어려|움을| 겪|고| 있습니다|.

|5|.| **|성| inequality|**|:| 성|별|에| 따른| 임|금| 격|차|,| 직|장| 내| 성|희|롱|,| 경|력| 단|절| 등| 성|불|평|등| 문제가| 여|전히| 존재|하고| 있습니다|.| 여성|들이| 사회|에서| 차|별|받|는| 경우|가| 많|아| 이를| 해결|하기| 위한| 노|력이| 필요|합니다|.

|6|.| **|정|신| 건강| 문제|**|:| 현대| 사회|의| 스트|레스|와| 경쟁|으로| 인해| 정신| 건강| 문제가| 증가|하고| 있습니다|.| 우|울

In [36]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_c

In [37]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'service_tier': 'default'}, id='run--d6d35485-65c8-4d99-bdad-162cc456e43d', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'index': 0, 'type': 'tool_call_chunk'}])

In [38]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-08-12 02:00:01 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'service_tier': 'default'}, id='run--d6d35485-65c8-4d99-bdad-162cc456e43d', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_1b2Hz2PSnvqb6syCuOfpOprC', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/S

In [39]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 지금| |202|5|년| |8|월| |12|일| |02|시| |00|분| |01|초|입니다|.||